In [1]:
#Clustering is done to form distinct groups having similar observations 
#It is an unsupervised learning method when label are mentioned
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('kmeans').getOrCreate()

In [3]:
from pyspark.ml.clustering import KMeans

In [4]:
data=spark.read.format('libsvm').load('sample_kmeans_data.txt')

In [6]:
data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|           (3,[],[])|
|  1.0|(3,[0,1,2],[0.1,0...|
|  2.0|(3,[0,1,2],[0.2,0...|
|  3.0|(3,[0,1,2],[9.0,9...|
|  4.0|(3,[0,1,2],[9.1,9...|
|  5.0|(3,[0,1,2],[9.2,9...|
+-----+--------------------+



In [7]:
kmeans=KMeans().setK(2).setSeed(1)

In [8]:
model=kmeans.fit(data)

In [11]:
#within set sum of sqaured error
wssse=model.computeCost(data)

In [12]:
wssse

0.11999999999994547

In [16]:
#See the cluster centers 
centers=model.clusterCenters()

In [17]:
centers

[array([0.1, 0.1, 0.1]), array([9.1, 9.1, 9.1])]

In [19]:
result=model.transform(data)

In [20]:
result.show()
# Can try with more number of clusters 
# in the prediction colums it specifies the cluster to which each observations belongs

+-----+--------------------+----------+
|label|            features|prediction|
+-----+--------------------+----------+
|  0.0|           (3,[],[])|         0|
|  1.0|(3,[0,1,2],[0.1,0...|         0|
|  2.0|(3,[0,1,2],[0.2,0...|         0|
|  3.0|(3,[0,1,2],[9.0,9...|         1|
|  4.0|(3,[0,1,2],[9.1,9...|         1|
|  5.0|(3,[0,1,2],[9.2,9...|         1|
+-----+--------------------+----------+



In [21]:
# KMeans clustring on a bigger dataset 

In [23]:
data=spark.read.csv('seeds_dataset.csv' ,inferSchema=True, header=True)

In [25]:
data.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)



In [26]:
from pyspark.ml.feature import VectorAssembler

In [27]:
data.columns

['area',
 'perimeter',
 'compactness',
 'length_of_kernel',
 'width_of_kernel',
 'asymmetry_coefficient',
 'length_of_groove']

In [30]:
assembler=VectorAssembler(inputCols=data.columns, outputCol='features')

In [32]:
finaldata=assembler.transform(data)

Exception ignored in: <function JavaWrapper.__del__ at 0x0000017B28D8C268>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pyspark\ml\wrapper.py", line 40, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'VectorAssembler' object has no attribute '_java_obj'
Exception ignored in: <function JavaWrapper.__del__ at 0x0000017B28D8C268>
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\pyspark\ml\wrapper.py", line 40, in __del__
    if SparkContext._active_spark_context and self._java_obj is not None:
AttributeError: 'VectorAssembler' object has no attribute '_java_obj'


In [36]:
finaldata.printSchema()

root
 |-- area: double (nullable = true)
 |-- perimeter: double (nullable = true)
 |-- compactness: double (nullable = true)
 |-- length_of_kernel: double (nullable = true)
 |-- width_of_kernel: double (nullable = true)
 |-- asymmetry_coefficient: double (nullable = true)
 |-- length_of_groove: double (nullable = true)
 |-- features: vector (nullable = true)



In [37]:
# Scaling the data 
from pyspark.ml.feature import StandardScaler

In [55]:
scaler=StandardScaler(inputCol='features', outputCol='Scaled features')

In [40]:
#since we are using only feautures column for our model (Clustering - Unsupervised leanring)
scaler_model=scaler.fit(finaldata)

In [42]:
finaldata=scaler_model.transform(finaldata)

In [43]:
finaldata.head()

Row(area=15.26, perimeter=14.84, compactness=0.871, length_of_kernel=5.763, width_of_kernel=3.312, asymmetry_coefficient=2.221, length_of_groove=5.22, features=DenseVector([15.26, 14.84, 0.871, 5.763, 3.312, 2.221, 5.22]), Scaled features=DenseVector([5.2445, 11.3633, 36.8608, 13.0072, 8.7685, 1.4772, 10.621]))

In [44]:
kmeans=KMeans(featuresCol='Scaled features', k=3)

In [45]:
model=kmeans.fit(finaldata)

In [46]:
wssse=model.computeCost(finaldata)

In [47]:
wssse


428.60820118716356

In [50]:
centers=model.clusterCenters()

In [51]:
centers

[array([ 4.07497225, 10.14410142, 35.89816849, 11.80812742,  7.54416916,
         3.15410901, 10.38031464]),
 array([ 6.35645488, 12.40730852, 37.41990178, 13.93860446,  9.7892399 ,
         2.41585013, 12.29286107]),
 array([ 4.96198582, 10.97871333, 37.30930808, 12.44647267,  8.62880781,
         1.80061978, 10.41913733])]

In [53]:
# To see the final predictions 
model.transform(finaldata).select('prediction').show()

+----------+
|prediction|
+----------+
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         1|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         2|
|         0|
+----------+
only showing top 20 rows

